#Task and Use-Case 
We are trying to predict the amount of alchohol people drink during the workweek based on their sex, age, mother's education, fathers education, Mother's job, father's job, main guardian, class failures, romantic status, and family relations. 

A classifier like this could be incredibly helpful because it could help get people the mental health services they need before turning to alcohol to drown out their problems, especially for students, which is who this data was collected from

This model could be deployed in Universities across the world to help health centers reach out to potential at-risk individuals from falling into an addiction

For this task a classifier with a prediction rate of 

In [12]:
import numpy as np
import pandas as pd

#X = np.load("Telecust1.csv")
#X= np.genfromtxt('Telecust1.csv',delimiter=',',dtype=None)
#print(X)
df = pd.read_csv('data.csv')
#print(df[["sex","age","Medu","Fedu","Mjob","Fjob","guardian","failures","romantic","famrel"]])

     sex  age  Medu  Fedu      Mjob      Fjob guardian  failures romantic  \
0      F   18     4     4   at_home   teacher   mother         0       no   
1      F   17     1     1   at_home     other   father         0       no   
2      F   15     1     1   at_home     other   mother         3       no   
3      F   15     4     2    health  services   mother         0      yes   
4      F   16     3     3     other     other   father         0       no   
...   ..  ...   ...   ...       ...       ...      ...       ...      ...   
1039   F   19     2     3  services     other   mother         1       no   
1040   F   18     3     1   teacher  services   mother         0       no   
1041   F   18     1     1     other     other   mother         0       no   
1042   M   17     3     1  services  services   mother         0       no   
1043   M   18     3     2  services     other   mother         0       no   

      famrel  
0          4  
1          5  
2          4  
3          3  


#Data Information

In [24]:
print(df[["sex","age","Medu","Fedu","Mjob","Fjob","guardian","failures","romantic","famrel"]].describe())

               age         Medu         Fedu     failures       famrel
count  1044.000000  1044.000000  1044.000000  1044.000000  1044.000000
mean     16.726054     2.603448     2.387931     0.264368     3.935824
std       1.239975     1.124907     1.099938     0.656142     0.933401
min      15.000000     0.000000     0.000000     0.000000     1.000000
25%      16.000000     2.000000     1.000000     0.000000     4.000000
50%      17.000000     3.000000     2.000000     0.000000     4.000000
75%      18.000000     4.000000     3.000000     0.000000     5.000000
max      22.000000     4.000000     4.000000     3.000000     5.000000


In [13]:
#the base Binary Logistic Regression class
class BinaryLogisticRegressionBase:
    # private:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        return 'Base Binary Logistic Regression Object, Not Trainable'
    
    # convenience, private and static:
    @staticmethod
    def _sigmoid(theta):
        return 1/(1+np.exp(-theta)) 
    
    @staticmethod
    def _add_intercept(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    # public:
    def predict_proba(self, X, add_intercept=True):
        # add bias term if requested
        Xb = self._add_intercept(X) if add_intercept else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction

In [14]:
#Binary Logistic regression, inheriting from the base
class BinaryLogisticRegression(BinaryLogisticRegressionBase):
    #private:
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    def _get_gradient(self,X,y):
        # programming \sum_i (yi-g(xi))xi
        gradient = np.zeros(self.w_.shape) # set gradient to zero
        for (xi,yi) in zip(X,y):
            # the actual update inside of sum
            gradi = (yi - self.predict_proba(xi,add_intercept=False))*xi 
            # reshape to be column vector and add to gradient
            gradient += gradi.reshape(self.w_.shape) 
        
        return gradient/float(len(y))
       
    # public:
    def fit(self, X, y):
        Xb = self._add_intercept(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 


In [15]:
from scipy.special import expit
#Vector binary logistic regression
class VectorBinaryLogisticRegression(BinaryLogisticRegression):
    # inherit from our previous class to get same functionality
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # but overwrite the gradient calculation
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_intercept=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        return gradient.reshape(self.w_.shape)


Creating 1 vs All logistic Regression

In [16]:
#ONE VS ALL LOGISTIC REGRESSION
#this takes each class and creates a binary classifier for it, and then runs each instance through all binary classifiers and choose the highest one
class LogisticRegression:
    def __init__(self, eta, iterations=20):
        self.eta = eta
        self.iters = iterations
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.unique(y) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = [] # will fill this array with binary classifiers
        
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval) # create a binary problem
            # train the binary classifier for this class
            blr = VectorBinaryLogisticRegression(self.eta,
                                                 self.iters)
            blr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(blr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for blr in self.classifiers_:
            probs.append(blr.predict_proba(X)) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return self.unique_[np.argmax(self.predict_proba(X),axis=1)] # take argmax along row
    
lr = LogisticRegression(0.1,1500)
print(lr)

Untrained MultiClass Logistic Regression Object


Using 1 vs All Logistic Regression

In [23]:
#importing these specific split and metric functions
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score



#using the test/train split function
X = df[["sex","age","Medu","Fedu","Mjob","Fjob","guardian","failures","romantic","famrel"]] # modify 
y = df["Dalc"] #pick a category that can be output
#print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.8, test_size=0.2, random_state=0)
print(X_train)
print(y_train)
logReg = LogisticRegression(0.1,500) #eta and iterations
#print(logReg)
logReg.fit(X_train,y_train)
#print(logReg)
yhat = lr.predict(X_test)
print("Accuracy of: ",accuracy_score(y_test,yhat))

     sex  age  Medu  Fedu      Mjob      Fjob guardian  failures romantic  \
145    F   15     1     1     other  services   father         0       no   
590    M   16     3     3  services     other   mother         0      yes   
482    F   15     4     2     other     other   mother         0       no   
251    M   16     3     3   at_home     other    other         0       no   
630    F   17     1     1   at_home     other   mother         0      yes   
...   ..  ...   ...   ...       ...       ...      ...       ...      ...   
1033   M   17     2     3     other  services   father         0       no   
763    M   18     4     4     other     other   father         0       no   
835    M   16     1     1   at_home  services   mother         0      yes   
559    M   16     2     1   at_home     other   mother         1      yes   
684    F   17     4     2   teacher  services   mother         0       no   

      famrel  
145        4  
590        4  
482        5  
251        5  


TypeError: can't multiply sequence by non-int of type 'float'

Stochastic Logistic Regression

In [ ]:
#i think it will work if we inherit logistic regression instead of binary logistic regression
class StochasticLogisticRegression(LogisticRegression):
    # stochastic gradient calculation 
    def _get_gradient(self,X,y):
        
        # grab a subset of samples in a mini-batch
        # and calculate the gradient according to the small batch only
        mini_batch_size = 16
        idxs = np.random.choice(len(y), mini_batch_size)
        
        ydiff = y[idxs]-self.predict_proba(X[idxs],add_bias=False).ravel() # get y difference (now scalar)
        gradient = np.mean(X[idxs] * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
        
        return gradient
    

Using Stochastic Logistic Regression

In [ ]:
slr = StochasticLogisticRegression(eta=0.01, iterations=300, C=0.001)
slr.fit(X_train,y_train)

yhat = slr.predict(X_test)
print(slr)
print('Accuracy of: ',accuracy_score(y_test,yhat))  

Setting up a vectorized version of Newton's update method, it runs significantly faster 

In [ ]:
from numpy.linalg import pinv
#made it Logistic Regression instead of Binary Logistic Regression
class HessianLogisticRegression(LogisticRegression):
    # just overwrite gradient function
    def _get_gradient(self,X,y):
        g = self.predict_proba(X,add_bias=False).ravel() # get sigmoid value for all classes
        hessian = X.T @ np.diag(g*(1-g)) @ X - 2 * self.C # calculate the hessian

        ydiff = y-g # get y difference
        gradient = np.sum(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        gradient[1:] += -2 * self.w_[1:] * self.C
        
        return pinv(hessian) @ gradient
       
       


#Using Hessian Logistic Regression (Vectorized Newton)

In [ ]:
hlr = HessianLogisticRegression(eta=1.0,
                                      iterations=4,
                                      C=0.001) # note that we need only a few iterations here

hlr.fit(X_train,y_train)
yhat = hlr.predict(X_test)
print(hlr)
print('Accuracy of: ',accuracy_score(y_test,yhat))

#Using Gradient Decent

In [ ]:
class 

#running code and choosing an optimazation type

In [ ]:
##this may need to go into the logistic regression class as different get gradient functions
##accept a parameter that decides which gradient function to use